In [40]:
import os
from ultralytics import YOLO
import cv2

In [41]:
model = YOLO(r'C:\Users\BM MONEY\runs\detect\train7\weights\best.pt')

In [42]:
video_path = 'videoplayback.mp4'

In [43]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"Could not open video: {video_path}")

In [44]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [45]:
output_path = 'output_video.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

In [46]:
# Track objects with IDs
no_helmet_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run inference with tracking
    results = model.track(frame, conf=0.25, tracker="botsort.yaml", persist=True)
    
    # Create a copy of the frame for custom annotations
    annotated_frame = frame.copy()
    
    # Update no-helmet IDs
    current_no_helmet_ids = set()
    for box in results[0].boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        track_id = int(box.id) if box.id is not None else None
        if track_id is None:
            continue
        
        # Get original head bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        # Expand bounding box to cover body
        head_width = x2 - x1
        head_height = y2 - y1
        
        # Assume body is ~3-4x head height downward and 2x head width
        body_height = int(head_height * 4)
        body_width = int(head_width * 2)
        
        # New coordinates
        new_x1 = max(0, x1 - head_width // 2)  # Extend left
        new_x2 = min(width, x2 + head_width // 2)  # Extend right
        new_y1 = max(0, y1)  # Keep top at head
        new_y2 = min(height, y1 + body_height)  # Extend downward
        
        # Draw expanded bounding box
        color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Green for With Helmet, Red for Without Helmet
        cv2.rectangle(annotated_frame, (new_x1, new_y1), (new_x2, new_y2), color, 2)
        cv2.putText(annotated_frame, f"ID: {track_id} {'No Helmet' if cls == 1 else 'Helmet'}", 
                    (new_x1, new_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Handle non-helmeted bikers
        if cls == 1 and conf > 0.5:
            print(f"Person without helmet detected! Track ID: {track_id}")
            current_no_helmet_ids.add(track_id)
            
            # Draw red block (filled rectangle) over the body
            red_block_alpha = 0.4  # Transparency for red block
            overlay = annotated_frame.copy()
            cv2.rectangle(overlay, (new_x1, new_y1), (new_x2, new_y2), (0, 0, 255), -1)  # Filled red
            cv2.addWeighted(overlay, red_block_alpha, annotated_frame, 1 - red_block_alpha, 0, annotated_frame)
    
    # Update no_helmet_ids
    no_helmet_ids = no_helmet_ids & current_no_helmet_ids
    no_helmet_ids.update(current_no_helmet_ids)
    
    # Global warning for non-helmeted bikers
    if no_helmet_ids:
        cv2.putText(annotated_frame, "No Helmet Detected!", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Write frame to output video
    out.write(annotated_frame)
    
    # Display frame
    cv2.imshow('Helmet Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Output video saved at: {output_path}")


0: 384x640 1 With Helmet, 1 Without Helmet, 60.6ms
Speed: 5.7ms preprocess, 60.6ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 With Helmet, 1 Without Helmet, 58.7ms
Speed: 5.5ms preprocess, 58.7ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 With Helmets, 28.0ms
Speed: 3.0ms preprocess, 28.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Without Helmet, 27.8ms
Speed: 3.4ms preprocess, 27.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 With Helmet, 27.8ms
Speed: 3.0ms preprocess, 27.8ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 With Helmet, 27.9ms
Speed: 3.2ms preprocess, 27.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.8ms
Speed: 2.8ms preprocess, 27.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.9ms
Speed